In [1]:
import torch
from model import FaceDetector
from dataset import JSdataset
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn

import os
import tqdm

import time
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter



In [2]:
torch.manual_seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


batch_size = 200
epoch = 0
learning_rate = 1e-3
print_every = 10
evaluate_every = 10
out_dir = "./out"
total_epoch=5


logger = SummaryWriter(os.path.join(out_dir, 'log'))

In [3]:
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [68]:
def train(model, optimizer, criterion, train_loader, test_loader, total_epoch):
    
    test_images, test_labels = next(iter(test_loader))
    test_images = test_images.to(device)
    test_labels = test_labels.type(torch.FloatTensor)

    model.train()

    it = 0

    t0b = time.time()
    for epoch in range(total_epoch):
        for images, labels in train_loader:
            model.train()
            images = images.to(device)
            labels = labels.type(torch.FloatTensor)
            labels = labels.to(device)

            loss = 0

            #initialize gradient
            optimizer.zero_grad()
            #forward propagation
            output = model(images)
            #calculate loss
            loss = criterion(output.squeeze(1),labels)
            #backward proopagation
            loss.backward()
            #update parameters
            optimizer.step()


            
            with torch.no_grad():
                model.eval()
                prediction = get_pred(model, test_images)
                accuracy = get_accuracy(prediction.squeeze(),test_labels)
                print(accuracy)
                
            loss_dicts = {"Train Loss": loss, "Test accuracy": accuracy}
            logger.add_scalars("train_loss",loss_dicts,it)                 
                
               

            if print_every > 0 and (it % print_every) == 0:
                print('[Epoch %02d] loss: %.4f it %03d, time%.3f' % (epoch,loss, it, time.time()-t0b))
            
            it+=1

In [69]:
def get_pred(pytorch_model,x):
    #x = torch.Tensor(x).to(device)
    output = pytorch_model(x)
    output = torch.round(output)
    output = output.detach().cpu()
    return output    

def get_accuracy(pred,label):
    return torch.sum(pred == label).item()/len(label)

In [70]:
train_dataset = JSdataset('/data/js_detection/dataset',train=True,transform =transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)),
    transforms.Normalize((0.5244, 0.5401, 0.5397),(0.2569, 0.2638, 0.2730))
]))

test_dataset = JSdataset('/data/js_detection/dataset',train=False,transform =transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)),
    transforms.Normalize((0.5244, 0.5401, 0.5397),(0.2569, 0.2638, 0.2730))
]))

train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

In [71]:
model = FaceDetector()
model.to(device)

optimizer = optim.Adagrad(model.parameters(), lr=learning_rate)
criterion = torch.nn.BCELoss()

In [72]:
train(model=model, optimizer=optimizer, criterion=criterion, train_loader=train_loader,test_loader=test_loader,total_epoch=total_epoch)

0.52
[Epoch 00] loss: 0.6941 it 000, time3.097
0.6
0.52
0.85
0.725
0.89
0.875
0.88
0.805
0.965
0.95
[Epoch 01] loss: 0.1558 it 010, time36.659
0.98
0.96
1.0
1.0
1.0
1.0
0.97
0.895
0.99
1.0
[Epoch 02] loss: 0.0573 it 020, time69.800
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
[Epoch 03] loss: 0.0225 it 030, time103.562
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
[Epoch 04] loss: 0.0095 it 040, time136.375
1.0
1.0
1.0
1.0


In [75]:
torch.save(model.state_dict(),'/data/js_detection/out/parameters.pth')